# ✅ Smart Coral Merger with Team Number Matching Fix
This version improves data integrity by handling misspelled team numbers in 7034 using fuzzy matching and anchoring matches to accurate DCMP data.

In [7]:

import pandas as pd
from itertools import combinations, product
from difflib import get_close_matches

# Load CSVs while treating "none" as a valid string
na_vals = ["", "NA", "N/A"]
dcmp_df = pd.read_csv("2025 WCMP Data accuracy - 1425 dat.csv", na_values=na_vals, keep_default_na=False)
csv7034_df = pd.read_csv("2025 WCMP Data accuracy - 7034 dat.csv", na_values=na_vals, keep_default_na=False)


In [8]:

# Normalize DCMP
dcmp_df['team_number'] = dcmp_df['team_key'].str.extract(r'(\d+)$').astype(float)
dcmp_df['match_key'] = dcmp_df['match_number'].astype(str) + '_' + dcmp_df['comp_level']

# Normalize 7034
csv7034_df = csv7034_df.dropna(subset=['Match #'])
csv7034_df['match_number'] = csv7034_df['Match #'].astype(int)
csv7034_df['match_key'] = csv7034_df['match_number'].astype(str) + '_qm'

# Fix team number typos using closest match from DCMP per match
def fix_team_number(row, dcmp_teams_by_match):
    if row['match_key'] not in dcmp_teams_by_match:
        return row['Team #']
    try:
        ref_teams = [str(int(t)) for t in dcmp_teams_by_match[row['match_key']]]
        best = get_close_matches(str(int(row['Team #'])), ref_teams, n=1, cutoff=0.7)
        return int(best[0]) if best else row['Team #']
    except:
        return row['Team #']

dcmp_teams_by_match = dcmp_df.groupby('match_key')['team_number'].unique().apply(list).to_dict()
csv7034_df['team_number'] = csv7034_df.apply(lambda r: fix_team_number(r, dcmp_teams_by_match), axis=1)
csv7034_df['alliance'] = csv7034_df['Robot'].str[0].map({'R': 'red', 'B': 'blue'})
csv7034_df = csv7034_df.drop_duplicates(subset=["match_key", "alliance", "team_number"])



In [9]:



def extract_alliance_teams(df):
    alliances = {}
    for _, row in df.iterrows():
        key = f"{row['match_number']}_{row['comp_level']}"
        if key not in alliances:
            alliances[key] = {
                'red': [row['r1'], row['r2'], row['r3']],
                'blue': [row['b1'], row['b2'], row['b3']],
            }
    return alliances

match_alliances = extract_alliance_teams(dcmp_df)

def assign_alliance(row):
    key = row['match_key']
    team_key = row['team_key']
    alliance = match_alliances.get(key, {})
    if 'red' in alliance and team_key in alliance['red']:
        return 'red'
    elif 'blue' in alliance and team_key in alliance['blue']:
        return 'blue'
    return None

dcmp_df['alliance'] = dcmp_df.apply(assign_alliance, axis=1)


In [10]:

def find_best_cell_level_combo(dcmp_rows, csv_rows, target, prefix):
    team_data = {}
    for row in dcmp_rows + csv_rows:
        team_number = row['team_number']
        if team_number not in team_data:
            team_data[team_number] = {}
        if f'{prefix}_L1' in row:
            team_data[team_number]['dcmp'] = row
        else:
            team_data[team_number]['7034'] = row

    best_diff = float('inf')
    best_combo = None

    for team_set in combinations(team_data.keys(), 3):
        robot_mix_options = list(product(['dcmp', '7034'], repeat=4))
        all_mix_combos = product(robot_mix_options, repeat=3)

        for robot_cell_choices in all_mix_combos:
            combo_rows = []
            valid = True
            total = 0

            for team, mix in zip(team_set, robot_cell_choices):
                sources = team_data[team]
                row = {}
                for i, lvl in enumerate(['L1', 'L2', 'L3', 'L4']):
                    dcmp_col = f'{prefix}_L{lvl[-1]}'
                    alt_col = f'{prefix}L{lvl[-1]}'
                    source = mix[i]
                    if source in sources:
                        src_row = sources[source]
                        val = src_row.get(dcmp_col) if source == 'dcmp' else src_row.get(alt_col, 0)
                        row[dcmp_col] = val
                        total += val
                    else:
                        valid = False
                        break
                row['team_number'] = team
                if not valid:
                    break
                combo_rows.append(row)

            if valid and abs(total - target) < best_diff:
                best_diff = abs(total - target)
                best_combo = combo_rows

    return best_combo


In [11]:

results = []

for match_key in dcmp_df['match_key'].unique():
    for alliance in ['red', 'blue']:
        dcmp_rows = list(dcmp_df[(dcmp_df['match_key'] == match_key) & (dcmp_df['alliance'] == alliance)].to_dict('records'))
        csv_rows = list(csv7034_df[(csv7034_df['match_key'] == match_key) & (csv7034_df['alliance'] == alliance)].to_dict('records'))
        if not dcmp_rows:
            continue

        target_auto = next((r['ba_autoCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_autoCoralCount'))), 0)
        target_tele = next((r['ba_teleopCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_teleopCoralCount'))), 0)

        team_info = {r['team_number']: r for r in dcmp_rows}
        best_auto = find_best_cell_level_combo(dcmp_rows, csv_rows, target_auto, 'a') or []
        best_tele = find_best_cell_level_combo(dcmp_rows, csv_rows, target_tele, 't') or []

        merged = {}

        for row in best_auto:
            tn = row['team_number']
            if tn not in merged:
                merged[tn] = team_info.get(tn, next((r for r in csv_rows if r['team_number'] == tn), {})).copy()
                merged[tn]['match_key'] = match_key
                merged[tn]['alliance'] = alliance
                merged[tn]['team_number'] = tn
                merged[tn]['team_key'] = merged[tn].get('team_key', f"frc{int(tn)}")
            for k, v in row.items():
                if k.startswith('a_'):
                    merged[tn][k] = v
            merged[tn]['selected_for_auto'] = 'auto'

        for row in best_tele:
            tn = row['team_number']
            if tn not in merged:
                merged[tn] = team_info.get(tn, next((r for r in csv_rows if r['team_number'] == tn), {})).copy()
                merged[tn]['match_key'] = match_key
                merged[tn]['alliance'] = alliance
                merged[tn]['team_number'] = tn
                merged[tn]['team_key'] = merged[tn].get('team_key', f"frc{int(tn)}")
            for k, v in row.items():
                if k.startswith('t_'):
                    merged[tn][k] = v
            merged[tn]['selected_for_teleop'] = 'teleop'

        for row in merged.values():
            results.append(row)

df_final = pd.DataFrame(results)
df_final['match_number'] = pd.to_numeric(df_final['match_number'], errors='coerce').fillna(-1).astype(int)
df_final['alliance'] = pd.Categorical(df_final['alliance'], categories=['red', 'blue'], ordered=True)
df_final = df_final.sort_values(by=['match_number', 'alliance', 'team_number'])

df_final.to_csv("smart_cell_level_corals_fixed_teams.csv", index=False)
print("✅ Saved to smart_cell_level_corals_fixed_teams.csv")


✅ Saved to smart_cell_level_corals_fixed_teams.csv
